# North Korea Missile Test

## 0. Importing libraries

In [1]:
import math
import matplotlib
matplotlib.use('Agg')
import pandas as pd
# import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn.datasets as datasets
import csv
import lightgbm as lgb

from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


## 1. Loading the data

In [2]:
df = pd.read_excel('./north_korea_missile_test_database.xlsx')
successCount=0
for indexDF, rowDF in df.iterrows():
    if (df.at[indexDF, 'Test Outcome'] =='Success'):
        successCount = successCount + 1
print ('Number of samples: ', df.shape[0])
print ('Number of features: ', df.shape[1])
print ('Number of successful launches: ', successCount)
df.keys()

Number of samples:  117
Number of features:  19
Number of successful launches:  88


Index(['F1', 'Date', 'Date Entered/Updated', 'Launch Time (UTC)',
       'Missile Name', 'Missile Type', 'Launch Agency/Authority',
       'Facility Name', 'Facility Location', 'Other Name', 'Facility Latitude',
       'Facility Longitude', 'Landing Location', 'Apogee',
       'Distance Travelled', 'Confirmation Status', 'Test Outcome',
       'Additional Information', 'Source(s)'],
      dtype='object')

## 2. Displaying the header

In [3]:
df.head(5)

,F1,Date,Date Entered/Updated,Launch Time (UTC),Missile Name,Missile Type,Launch Agency/Authority,Facility Name,Facility Location,Other Name,Facility Latitude,Facility Longitude,Landing Location,Apogee,Distance Travelled,Confirmation Status,Test Outcome,Additional Information,Source(s)
0,1,1984-04-09,2016-12-23,NaN,Scud-B,SRBM,NaN,Tonghae Satellite Launching Ground,"Hwadae County, North Hamgyong Province",Musudan-ri,40.85,129.667,Unknown,Unknown,Unknown,Confirmed,Success,First known test by North Korea of a missile w...,http://www.nti.org/media/pdfs/north_korea_miss...
1,2,1984-04-09,2016-12-23,NaN,Scud-B,SRBM,NaN,Tonghae Satellite Launching Ground,"Hwadae County, North Hamgyong Province",Musudan-ri,40.85,129.667,Unknown,Unknown,Unknown,Confirmed,Success,NaN,http://www.nti.org/media/pdfs/north_korea_miss...
2,3,1984-04-09,2016-12-23,NaN,Scud-B,SRBM,NaN,Tonghae Satellite Launching Ground,"Hwadae County, North Hamgyong Province",Musudan-ri,40.85,129.667,Unknown,200 km,Unknown,Confirmed,Success,NaN,http://www.astronautix.com/g/gitdaeryung.html;...
3,4,1984-09-01,2016-12-23,NaN,Scud-B,SRBM,NaN,Tonghae Satellite Launching Ground,"Hwadae County, North Hamgyong Province",Musudan-ri,40.85,129.667,Unknown,Unknown,Unknown,Confirmed,Failure,NaN,"Joseph S. Bermudez, ""A History of Ballistic Mi..."
4,5,1984-09-01,2016-12-23,NaN,Scud-B,SRBM,NaN,Tonghae Satellite Launching Ground,"Hwadae County, North Hamgyong Province",Musudan-ri,40.85,129.667,Unknown,200 km,Unknown,Confirmed,Failure,NaN,http://www.astronautix.com/g/gitdaeryung.html;...


## 3. Organizing the features

In [4]:
unusable_features = ['F1', 
                    'Date Entered/Updated', 
                    'Missile Name',
                    'Launch Agency/Authority',
                    'Confirmation Status',
                    'Additional Information',
                    'Apogee',
                    'Distance Travelled', 
                    'Landing Location',
                    'Source(s)'
                    ]

categorical_features = ['Missile Type',
                         'Facility Name',
                         'Facility Location',
                         'Other Name',        
                        ]

features_missing_values = ['Launch Time (UTC)',
                           'Facility Longitude',
                           'Facility Latitude'
                          ]

features_missing_str = ['Other Name']


time_shaped_features = ['Date',
                        'Launch Time (UTC)'
                       ]


## 4. Cleaning the bad features

In [5]:
for bad_feature in unusable_features:
    df = df.drop(bad_feature, axis = 1)
    
for indexDF, rowDF in df.iterrows():
    if (df.at[indexDF, 'Test Outcome'] == 'Unknown'):
        df = df.drop(indexDF, axis = 0)

In [6]:
for bad_feature in features_missing_str:
    for indexDF, rowDF in df.iterrows():
        if (pd.isnull(df.at[indexDF, bad_feature])):
            df.at[indexDF,bad_feature] = 'Other'

In [7]:
for indexDF, rowDF in df.iterrows():
    if(df.at[indexDF, 'Test Outcome'] == 'Success'):
        df.at[indexDF,'Test Outcome'] = True
    else:
        df.at[indexDF,'Test Outcome'] = False

In [8]:
import datetime
import time

df['Date'] = pd.to_timedelta(df['Date'], errors='coerce')
df['Launch Time (UTC)'] = pd.to_timedelta(df['Launch Time (UTC)'].astype(str), errors='coerce')
df['Launch Time (UTC)'] = df['Launch Time (UTC)'].dt.total_seconds()
df['Date'] = df['Date'].dt.total_seconds()

In [9]:
for missing_feature in features_missing_values:
    for indexDF, rowDF in df.iterrows():
        if (df.at[indexDF, missing_feature] == 'Unknown'):
            df.at[indexDF,missing_feature] = None

for missing_feature in features_missing_values:
    mu = np.mean(df[missing_feature])
    sigma = np.std(df[missing_feature])
    for indexDF, rowDF in df.iterrows():
        if (pd.isnull(df.at[indexDF, missing_feature])):
            df.at[indexDF,missing_feature] = np.random.normal(loc=mu, scale=sigma)

In [10]:

from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

for cat_feature in categorical_features:
    dummies = pd.get_dummies(df[cat_feature])
    df = pd.concat([df, dummies], axis=1)
    
for cat_feature in categorical_features:
    df = df.drop([cat_feature], axis=1)
    

## 5. Spliting the data

In [11]:
from sklearn.model_selection import train_test_split

y = df['Test Outcome']
X = df.drop(['Test Outcome'],1)

X, garboX, y, garboY = train_test_split(X, y, test_size=0.00)

trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.30)

## 6. Training the autoencoder

In [12]:
import matplotlib.pyplot as plt
from keras.layers import Input,Dense
from keras.models import Model,Sequential
from keras.wrappers.scikit_learn import KerasRegressor

np.random.seed(7)
input_dim=trainX.shape[1]
encoding_dim = 5  


input_data = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_data)
decoded = Dense(input_dim, activation='sigmoid')(encoded)
autoencoder = Model(input_data, decoded)
encoder = Model(input_data, encoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(trainX, trainX,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(testX, testX))

Train on 79 samples, validate on 35 samples
Epoch 1/100
79/79 [==============================] - 5s 62ms/step - loss: 346897888.0000 - val_loss: 342814528.0000
Epoch 2/100
79/79 [==============================] - 0s 76us/step - loss: 346897888.0000 - val_loss: 342814528.0000
Epoch 3/100
79/79 [==============================] - 0s 101us/step - loss: 346897856.0000 - val_loss: 342814528.0000
Epoch 4/100
79/79 [==============================] - 0s 114us/step - loss: 346897888.0000 - val_loss: 342814528.0000
Epoch 5/100
79/79 [==============================] - 0s 114us/step - loss: 346897920.0000 - val_loss: 342814528.0000
Epoch 6/100
79/79 [==============================] - 0s 88us/step - loss: 346897888.0000 - val_loss: 342814528.0000
Epoch 7/100
79/79 [==============================] - 0s 101us/step - loss: 346897856.0000 - val_loss: 342814528.0000
Epoch 8/100
79/79 [==============================] - 0s 76us/step - loss: 346897856.0000 - val_loss: 342814528.0000
Epoch 9/100
79/79 [=====

Epoch 71/100
79/79 [==============================] - 0s 76us/step - loss: 346897856.0000 - val_loss: 342814528.0000
Epoch 72/100
79/79 [==============================] - 0s 63us/step - loss: 346897888.0000 - val_loss: 342814528.0000
Epoch 73/100
79/79 [==============================] - 0s 76us/step - loss: 346897888.0000 - val_loss: 342814528.0000
Epoch 74/100
79/79 [==============================] - 0s 63us/step - loss: 346897888.0000 - val_loss: 342814528.0000
Epoch 75/100
79/79 [==============================] - 0s 63us/step - loss: 346897920.0000 - val_loss: 342814528.0000
Epoch 76/100
79/79 [==============================] - 0s 76us/step - loss: 346897888.0000 - val_loss: 342814528.0000
Epoch 77/100
79/79 [==============================] - 0s 63us/step - loss: 346897888.0000 - val_loss: 342814528.0000
Epoch 78/100
79/79 [==============================] - 0s 63us/step - loss: 346897856.0000 - val_loss: 342814528.0000
Epoch 79/100
79/79 [==============================] - 0s 76us/st

## 7. Encoding the features

In [13]:
enc_trainX = encoder.predict(trainX)
enc_testX = encoder.predict(testX)

## 8. Training the predictor

In [14]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
# fix random seed for reproducibility
np.random.seed(7)
# split into input (X) and output (Y) variables
input_dim=enc_trainX.shape[1]
output_dim=trainy.ndim
# create model
model = Sequential()
model.add(Dense(3,  input_shape=(input_dim,),  activation='sigmoid'))
model.add(Dense(output_dim, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Early stopping
callbacks = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]
# Fit the model
history = model.fit(x = enc_trainX, y = trainy,
                  epochs=100, 
                  batch_size=256,
                  shuffle=True,
#                   callbacks=callbacks,
                  validation_data=(enc_testX, testy))

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# evaluate the model
Tra_scores = model.evaluate(enc_trainX, trainy)
Val_scores = model.evaluate(enc_testX, testy)
tra_accuracy = Tra_scores[1]*100
val_accuracy = Val_scores[1]*100
accuracy_name = model.metrics_names[1]
print('\n Training', accuracy_name, ': ', tra_accuracy, '%')
print('\n Validation', accuracy_name, ': ', val_accuracy, '%')

Train on 79 samples, validate on 35 samples
Epoch 1/100
79/79 [==============================] - 0s 3ms/step - loss: 0.6171 - acc: 0.7468 - val_loss: 0.5854 - val_acc: 0.8286
Epoch 2/100
79/79 [==============================] - 0s 50us/step - loss: 0.6168 - acc: 0.7468 - val_loss: 0.5848 - val_acc: 0.8286
Epoch 3/100
79/79 [==============================] - 0s 63us/step - loss: 0.6164 - acc: 0.7468 - val_loss: 0.5842 - val_acc: 0.8286
Epoch 4/100
79/79 [==============================] - 0s 101us/step - loss: 0.6160 - acc: 0.7468 - val_loss: 0.5835 - val_acc: 0.8286
Epoch 5/100
79/79 [==============================] - 0s 88us/step - loss: 0.6155 - acc: 0.7468 - val_loss: 0.5829 - val_acc: 0.8286
Epoch 6/100
79/79 [==============================] - 0s 88us/step - loss: 0.6151 - acc: 0.7468 - val_loss: 0.5822 - val_acc: 0.8286
Epoch 7/100
79/79 [==============================] - 0s 88us/step - loss: 0.6147 - acc: 0.7468 - val_loss: 0.5815 - val_acc: 0.8286
Epoch 8/100
79/79 [=============

Epoch 63/100
79/79 [==============================] - 0s 63us/step - loss: 0.5938 - acc: 0.7468 - val_loss: 0.5476 - val_acc: 0.8286
Epoch 64/100
79/79 [==============================] - 0s 63us/step - loss: 0.5935 - acc: 0.7468 - val_loss: 0.5471 - val_acc: 0.8286
Epoch 65/100
79/79 [==============================] - 0s 51us/step - loss: 0.5932 - acc: 0.7468 - val_loss: 0.5466 - val_acc: 0.8286
Epoch 66/100
79/79 [==============================] - 0s 63us/step - loss: 0.5929 - acc: 0.7468 - val_loss: 0.5461 - val_acc: 0.8286
Epoch 67/100
79/79 [==============================] - 0s 76us/step - loss: 0.5926 - acc: 0.7468 - val_loss: 0.5456 - val_acc: 0.8286
Epoch 68/100
79/79 [==============================] - 0s 51us/step - loss: 0.5923 - acc: 0.7468 - val_loss: 0.5451 - val_acc: 0.8286
Epoch 69/100
79/79 [==============================] - 0s 63us/step - loss: 0.5921 - acc: 0.7468 - val_loss: 0.5446 - val_acc: 0.8286
Epoch 70/100
79/79 [==============================] - 0s 50us/step - 

C:\Users\hecto\Anaconda3\envs\tensorflow\lib\site-packages\matplotlib\figure.py:448: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [18]:
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
trainX_pca = pca.fit_transform(trainX)
testX_pca = pca.transform(testX)

In [19]:
model.fit(x = trainX_pca, y = trainy,
                  epochs=100, 
                  batch_size=256,
                  shuffle=True,
#                   callbacks=callbacks,
                  validation_data=(testX_pca, testy))

Train on 79 samples, validate on 35 samples
Epoch 1/100
79/79 [==============================] - 0s 3ms/step - loss: 0.5625 - acc: 0.7468 - val_loss: 0.4960 - val_acc: 0.8286
Epoch 2/100
79/79 [==============================] - 0s 76us/step - loss: 0.5625 - acc: 0.7468 - val_loss: 0.4958 - val_acc: 0.8286
Epoch 3/100
79/79 [==============================] - 0s 63us/step - loss: 0.5624 - acc: 0.7468 - val_loss: 0.4956 - val_acc: 0.8286
Epoch 4/100
79/79 [==============================] - 0s 76us/step - loss: 0.5623 - acc: 0.7468 - val_loss: 0.4954 - val_acc: 0.8286
Epoch 5/100
79/79 [==============================] - 0s 76us/step - loss: 0.5623 - acc: 0.7468 - val_loss: 0.4952 - val_acc: 0.8286
Epoch 6/100
79/79 [==============================] - 0s 101us/step - loss: 0.5622 - acc: 0.7468 - val_loss: 0.4950 - val_acc: 0.8286
Epoch 7/100
79/79 [==============================] - 0s 101us/step - loss: 0.5621 - acc: 0.7468 - val_loss: 0.4948 - val_acc: 0.8286
Epoch 8/100
79/79 [============

Epoch 63/100
79/79 [==============================] - 0s 76us/step - loss: 0.5603 - acc: 0.7468 - val_loss: 0.4887 - val_acc: 0.8286
Epoch 64/100
79/79 [==============================] - 0s 63us/step - loss: 0.5603 - acc: 0.7468 - val_loss: 0.4887 - val_acc: 0.8286
Epoch 65/100
79/79 [==============================] - 0s 63us/step - loss: 0.5602 - acc: 0.7468 - val_loss: 0.4886 - val_acc: 0.8286
Epoch 66/100
79/79 [==============================] - 0s 63us/step - loss: 0.5602 - acc: 0.7468 - val_loss: 0.4886 - val_acc: 0.8286
Epoch 67/100
79/79 [==============================] - 0s 63us/step - loss: 0.5602 - acc: 0.7468 - val_loss: 0.4885 - val_acc: 0.8286
Epoch 68/100
79/79 [==============================] - 0s 63us/step - loss: 0.5602 - acc: 0.7468 - val_loss: 0.4885 - val_acc: 0.8286
Epoch 69/100
79/79 [==============================] - 0s 76us/step - loss: 0.5602 - acc: 0.7468 - val_loss: 0.4884 - val_acc: 0.8286
Epoch 70/100
79/79 [==============================] - 0s 51us/step - 